# Progressive GAN

The code in this notebook is creadited to Jason Brownlee and was created by walking through his article [How to Train a Progressive Growing GAN in Keras for Synthesizing Faces](https://machinelearningmastery.com/how-to-train-a-progressive-growing-gan-in-keras-for-synthesizing-faces/)

This notebook uses the CelebA and cartoon datasets to synthesize images using a progressive GAN architecture. We found an informative article with a walkthrough of code and architecture setup to train progressive GANs.This article provided a lot of great explantion of the complex architecture and how to implement the custom layers that allow for the progressive learning. We used this architecture as a comparison to our model and the Info-GAN architecture.  
  
One of the main improvements that Progressive GANs provide is the incremental increase in the size of the images output by the generator. The training procedure will fine-tune a given output size and then slowly fade in a new model with a slightly larger resolution size. During this process all layers remain trainable which adds to the efficiency and perforamane of progressive GANs  
  
##### Note: throughout this notebook you will need to point the locations of the data and models to load/save to your local directory

### Data preprocessing

We trained the Progressive GAN on the CelebA dataset and a cartoon dataset. You can download the CelebA dataset [here](https://www.kaggle.com/jessicali9530/celeba-dataset)

Progressive GANs work best with small square images and since we are only concerned with generating faces, we can remove any background pixels using the [MTCNN](https://machinelearningmastery.com/how-to-train-a-progressive-growing-gan-in-keras-for-synthesizing-faces/) face detection model.

In [ ]:
!pip install mtcnn

In [ ]:
from keras import backend, constraints, initializers, layers, models as KM, optimizers
from math import sqrt
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from numpy import asarray, load, ones, savez_compressed, random, zeros
from os import listdir
from PIL import Image
from skimage.transform import resize

### Use Multi-Task Cascaded Convolutional Neural Network (MTCNN) to detect faces in dataset

In [ ]:
def load_image(filename):
    """Loads an image as an RGB numpy array
    
    Parameters
    ----------
    filename: str
        The name of the file from the source dataset

    Returns
    -------
    numpy.ndarray
        An array of pixel values from the input file image
    """
    image = Image.open(filename).convert('RGB')
    pixels = asarray(image)

    return pixels


def extract_face(model, pixels, required_size=(128, 128)):
    """Extracts face from loaded image with MTCNN and resizes to square
    
    Parameters
    ----------
    model: mtcnn.MTCNN
        The MTCNN model used to detect faces in input image
        
    pixels: numpy.ndarray
        Array of pixel values from input image
    
    required_size: tuple
        The resized image size

    Returns
    -------
    numpy.ndarray
        An array of the resized face detected from the image
    """

    faces = model.detect_faces(pixels)

    if len(faces) == 0:
        return None

    # Get the first face if more than one are detected in the image
    x1, y1, width, height = faces[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    face_pixels = pixels[y1:y2, x1:x2]

    image = Image.fromarray(face_pixels).resize(required_size)
    face_array = asarray(image)

    return face_array
 

def load_face_data(dataset_dir, num_faces):
    """Load in dataset and extract faces from all images
    
    Parameters
    ----------
    dataset_dir: str
        The directory that cotains the image dataset
        
    num_faces: int
        The number of faces wanted to train
    
    Returns
    -------
    numpy.ndarray
        An array of the faces for training
    """

    model = MTCNN()
    faces = list()

    for filename in listdir(dataset_dir):
        if filename.endswith('.csv'):
            continue

        pixels = load_image(dataset_dir + filename)
        face = extract_face(model, pixels)

        if face is None:
            continue

        faces.append(face)

        if len(faces) >= num_faces:
            break

    return asarray(faces)


#### Run preprocessing

1. Load dataset
2. Detect faces
3. Save preprocessed data

In [ ]:
dataset_directory = 'img_align_celeba/'
# dataset_directory = 'cartoonset10k/'
num_faces_for_training = 50000

all_faces = load_face_data(dataset_directory, num_faces_for_training)
print('Loaded: ', all_faces.shape)

savez_compressed('img_align_celeba_128.npz', all_faces)
# savez_compressed('cartoon.npz', all_faces)

Data can now be loaded in for training

In [ ]:
from numpy import load

data = load('img_align_celeba_128_5000.npz')
# data = load('cartoon.npz')

faces = data['arr_0']
print('Loaded: ', faces.shape)

In [ ]:
%matplotlib inline
pyplot.rcParams['figure.figsize'] = (15, 15)

def plot_faces(faces, n):
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(faces[i].astype('uint8'))
    pyplot.show()

print('Loaded: ', faces.shape)
plot_faces(faces, 10)

---

## Model Architecture

In this approach, each phase of growth is defined as its own model. For this to work properly, there are several custom layers that need to be implemented:  
1. PixelNormalization - used to normalize activations in generator  
2. MinibatchStdev - Summerizes batch statistics from images in discriminator  
3. WeightedSum - Controls the weighted sum of old and new layers during the phase-in process of training

### Pixel Normalization Layer:  


In progressive GANs, instead of using a batch normalization like many other GAN architectures, each pixel in the activations is normalized to unit length. [Progressive Growing of GANs for Improved Quality, Stability, and Variation](https://arxiv.org/abs/1710.10196) by Tero Karras, et al. defines this as pixelwise feature vector normalization. It's good to note that in this architecture we are only normalizing the generator model and not the discriminator.

This layer is used between each convolutional layer and activation function in the generator.


In [ ]:
class PixelNormalization(layers.Layer):
    """
    A custom Keras layer used to normalize the activation map in the generator
    ...

    Attributes
    ----------
    Layer : keras.layers.Layer
        Parent class used for implementing a custom Keras layer

    Methods
    -------
    call(inputs)
        Compute the normalized activations
        
    compute_output_shape(input_shape)
        Allows keras to do automatic shape inference. In this layer 
        the input shape is the same as the output shape
    """

    def __init__(self, **kwargs):
        super(PixelNormalization, self).__init__(**kwargs)


    def call(self, inputs):
        """Computes the normalized activations

        Parameters
        ----------
        inputs : numpy.ndarray
            array of the input activations
        
        Returns
        -------
        numpy.ndarray
            An array of the normalized activations
        """

        values = inputs**2.0
        mean_values = backend.mean(values, axis=-1, keepdims=True)
        mean_values += 1.0e-8
        l2 = backend.sqrt(mean_values)
        normalized = inputs / l2

        return normalized

    def compute_output_shape(self, input_shape):
        """Define the shape of the output

        Parameters
        ----------
        input_shape : tuple
            array of the input shape
        
        Returns
        -------
        tuple
            An array of output layer shape
        """
        return input_shape

### Minibatch Standard Deviation Layer:  

This custom layer summerizes a batch of activations in the output layer of discriminator. With this approach, the discriminator learns to better detect batches of fake samples from batches of real samples. The encoourages the generator to create batches of samples with realistic batch satatistics. 

  
The activation batch statistics are gathered by calculating the standard deviation of each pixel value in the activation and then calculating the avearage of that value. This results in a single channel activation map that is appened to the input list of activation maps.

In [ ]:
class MinibatchStdev(layers.Layer):
    """
    A custom Keras layer used to normalize the activation map in the generator
    ...

    Attributes
    ----------
    Layer : keras.layers.Layer
        Parent class used for implementing a custom Keras layer

    Methods
    -------
    call(inputs)
        Compute the batch activation statistics
        
    compute_output_shape(input_shape)
        Allows keras to do automatic shape inference. In this layer 
        the input shape is the same as the output shape
    """

    def __init__(self, **kwargs):
        super(MinibatchStdev, self).__init__(**kwargs)


    def call(self, inputs):
        """Compute the batch activation statistics

        Parameters
        ----------
        inputs : numpy.ndarray
            array of the input activations
        
        Returns
        -------
        numpy.ndarray
            An activation map with the statistic summary appended
        """
        mean = backend.mean(inputs, axis=0, keepdims=True)
        squ_diffs = backend.square(inputs - mean)
        mean_sq_diff = backend.mean(squ_diffs, axis=0, keepdims=True)
        mean_sq_diff += 1e-8
        stdev = backend.sqrt(mean_sq_diff)
        mean_pix = backend.mean(stdev, keepdims=True)
        shape = backend.shape(inputs)
        output = backend.tile(mean_pix, (shape[0], shape[1], shape[2], 1))

        combined = backend.concatenate([inputs, output], axis=-1)

        return combined

    def compute_output_shape(self, input_shape):
        """Added channel to output shape for batch statistics

        Parameters
        ----------
        input_shape : tuple
            tuple of the input shape
        
        Returns
        -------
        tuple
            tuple of output layer shape
        """
        input_shape = list(input_shape)
        input_shape[-1] += 1

        return tuple(input_shape)

### Weighted Sum Layer:  

This custom layer merges the activations from two input layers using a variable called alpha that controls how much to weight the first and second inputs. It is used in both the generator and discriminator during the growth phase of training when the model is transitioning from one image size to a new image size (e.g., from 4×4 to 8×8 pixels).

The alpha parameter is linearly scaled from 0.0 at the beginning to 1.0 at the end, allowing the output of the layer to transition from giving full weight to the old layers to giving full weight to the new layers (second input).

In [ ]:
class WeightedSum(layers.Add):
    """
    A custom Keras layer extending the Add merge layer used to 
    merge the activations of two input layers
    ...

    Attributes
    ----------
    Add : keras.layers.Add
        Parent class used for implementing a custom Keras Add merge layer
    
    alpha: used to linearly scale the weight on each of the two inputs

    Methods
    -------
    merge_fun(inputs)
        Compute the batch activation statistics
        
    compute_output_shape(input_shape)
        Allows keras to do automatic shape inference. In this layer 
        the input shape is the same as the output shape
    """

    def __init__(self, alpha=0.0, **kwargs):
        super(WeightedSum, self).__init__(**kwargs)
        self.alpha = backend.variable(alpha, name='ws_alpha')

    def _merge_function(self, inputs):
        # merges two input layers based on a linear alpha value
        assert (len(inputs) == 2)
        output = ((1.0 - self.alpha) * inputs[0]) + (self.alpha * inputs[1])

        return output

In [ ]:
def wasserstein_loss(y_true, y_hat):
    """Calculates wasserstein loss
    
    Parameters
    ----------
    y_true: Tensor
        The true labels
        
    y_hat: Tensor
        The predicted values
    
    Returns
    -------
    Tensor
        A tensor of loss values
    """
    return backend.mean(y_true * y_hat)

In [ ]:
def add_discriminator_block(src_model, n_input_layers=3):
    """Takes a trained discriminator model and creates a new growth model
    
    Parameters
    ----------
    src_model: keras.models.Model
        The old model prior to growth
        
    n_input_layers: int
        The number of input layers
    
    Returns
    -------
    list
        A list of two models - one growth model with two pathways and the WeightedSum layer, 
        and a copy of the src_model without the 1x1 layer and the WeightedSum layer
    """

    init = initializers.RandomNormal(stddev=0.02)
    const = constraints.max_norm(1.0)

    in_shape = list(src_model.input.shape)
    # print('in_shape: {}'.format(in_shape[-2].value))

    input_shape = (in_shape[-2]*2, in_shape[-2]*2, in_shape[-1])
    in_image = layers.Input(shape=input_shape)

    d = layers.Conv2D(128, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(in_image)
    d = layers.LeakyReLU(alpha=0.2)(d)

    d = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.AveragePooling2D()(d)
    block_new = d

    for i in range(n_input_layers, len(src_model.layers)):
        d = src_model.layers[i](d)

    model1 = KM.Model(in_image, d)

    model1.compile(loss=wasserstein_loss,
                   optimizer=optimizers.Adam(lr=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))

    downsample = layers.AveragePooling2D()(in_image)

    block_old = src_model.layers[1](downsample)
    block_old = src_model.layers[2](block_old)

    d = WeightedSum()([block_old, block_new])

    for i in range(n_input_layers, len(src_model.layers)):
        d = src_model.layers[i](d)

    model2 = KM.Model(in_image, d)

    model2.compile(loss=wasserstein_loss,
                   optimizer=optimizers.Adam(lr=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))

    return [model1, model2]

In [ ]:
def define_discriminator(n_phases, input_shape=(4,4,3)):
    """Defines base discriminator model
    
    Parameters
    ----------
    n_phases: int
        The number of phases the model with have (e.g., 4 - 4x4, 8x8, 16x16, 32x32)
        
    input_shape: tuple
        The tuple shape of the input
    
    Returns
    -------
    list
        A list of models for each of the N phases
    """

    init = initializers.RandomNormal(stddev=0.02)
    const = constraints.max_norm(1.0)

    model_list = list()
    in_image = layers.Input(shape=input_shape)

    d = layers.Conv2D(128, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(in_image)
    d = layers.LeakyReLU(alpha=0.2)(d)

    d = MinibatchStdev()(d)
    d = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)

    d = layers.Conv2D(128, (4,4), padding='same', kernel_initializer=init, kernel_constraint=const)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)

    d = layers.Flatten()(d)
    out_class = layers.Dense(1)(d)

    model = KM.Model(in_image, out_class)

    model.compile(loss=wasserstein_loss,
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))

    model_list.append([model, model])

    for i in range(1, n_phases):
        src_model = model_list[i - 1][0]
        models = add_discriminator_block(src_model)
        model_list.append(models)

    return model_list


In [ ]:
def add_generator_block(src_model):
    """Takes a trained geneartor model and creates a new growth model
    
    Parameters
    ----------
    src_model: keras.models.Model
        The old model prior to growth
    
    Returns
    -------
    list
        A list of two models - one growth model with two pathways and the WeightedSum layer, 
        and a copy of the src_model without the 1x1 layer and the WeightedSum layer
    """

    init = initializers.RandomNormal(stddev=0.02)
    const = constraints.max_norm(1.0)
    block_end = src_model.layers[-2].output
    upsampling = layers.UpSampling2D()(block_end)

    g = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(upsampling)
    g = PixelNormalization()(g)
    g = layers.LeakyReLU(alpha=0.2)(g)
    g = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    g = PixelNormalization()(g)
    g = layers.LeakyReLU(alpha=0.2)(g)

    out_image = layers.Conv2D(3, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(g)

    model1 = KM.Model(src_model.input, out_image)

    out_old = src_model.layers[-1]
    out_image2 = out_old(upsampling)

    merged = WeightedSum()([out_image2, out_image])
    model2 = models.Model(src_model.input, merged)

    return [model1, model2]

In [ ]:
def define_generator(latent_dim, n_phases, in_dim=4):
    """Defines base generator model

    Parameters
    ----------
    latent_dim: int
        The number of latent datapoints as input

    n_phases: int
        The number of phases the model with have (e.g., 4 - 4x4, 8x8, 16x16, 32x32)

    in_dim: int
        The input dimension

    Returns
    -------
    list
        A list of models for each of the N phases
    """

    init = initializers.RandomNormal(stddev=0.02)

    const = constraints.max_norm(1.0)
    model_list = list()

    in_latent = layers.Input(shape=(latent_dim,))

    g = layers.Dense(128 * in_dim * in_dim, kernel_initializer=init, kernel_constraint=const)(in_latent)
    g = layers.Reshape((in_dim, in_dim, 128))(g)

    g = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    g = PixelNormalization()(g)
    g = layers.LeakyReLU(alpha=0.2)(g)

    g = layers.Conv2D(128, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(g)
    g = PixelNormalization()(g)
    g = layers.LeakyReLU(alpha=0.2)(g)

    out_image = layers.Conv2D(3, (1,1), padding='same', kernel_initializer=init, kernel_constraint=const)(g)

    model = KM.Model(in_latent, out_image)
    model_list.append([model, model])

    for i in range(1, n_phases):
        src_model = model_list[i - 1][0]
        models = add_generator_block(src_model)
        model_list.append(models)

    return model_list

In [ ]:
def define_composite(discriminators, generators):
    """Defines the full composit model including the discrimnator and generator phases

    Parameters
    ----------
    discriminators: list
        list of discriminator phase models

    generators: list
        list of generator phase models

    Returns
    -------
    list
        A list of composit models for each of the N phases
    """
    model_list = list()

    for i in range(len(discriminators)):
        g_models, d_models = generators[i], discriminators[i]
        d_models[0].trainable = False
        model1 = models.Sequential()
        model1.add(g_models[0])
        model1.add(d_models[0])
        model1.compile(loss=wasserstein_loss,
                       optimizer=optimizers.Adam(lr=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))

        d_models[1].trainable = False
        model2 = models.Sequential()
        model2.add(g_models[1])
        model2.add(d_models[1])
        model2.compile(loss=wasserstein_loss,
                       optimizer=optimizers.Adam(lr=0.001, beta_1=0, beta_2=0.99, epsilon=10e-8))
        model_list.append([model1, model2])

    return model_list

In [ ]:
def load_real_samples(filename):
    """Loads real samples from training dataset

    Parameters
    ----------
    filename: str
        The filename of the sample

    Returns
    -------
    list
        A scaled version of the sample from [0, 255] to [-1, 1]
    """
    data = load(filename)
    X = data['arr_0'].astype('float32')
    X = (X - 127.5) / 127.5

    return X


def generate_real_samples(dataset, n_samples):
    """Selects random real samples form dataset

    Parameters
    ----------
    dataset: numpy.ndarray
        The source dataset

    Returns
    -------
    list
        A tuple of features and labels
    """
    ix = random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))

    return X, y


def generate_latent_points(latent_dim, n_samples):
    """Generate random points in latent space as input for the generator

    Parameters
    ----------
    latent_dim: int
        number of latent points per sample

    n_samples: int
        number of samples

    Returns
    -------
    numpy.ndarray
        A batch of inputs for network
    """

    x_input = random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)

    return x_input


def generate_fake_samples(generator, latent_dim, n_samples):
    """Generate fake samples with generator with labels

    Parameters
    ----------
    generator: keras.models.Model
        generator model

    latent_dim: int
        number of latent points per sample

    n_samples: int
        number of samples

    Returns
    -------
    tuple
        Fake image samples with class labels
    """

    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = -ones((n_samples, 1))

    return X, y


def update_fadein(models, step, n_steps):
    """Update the alpha values on each instance of WeightedSum layer

    Parameters
    ----------
    models: list
        list of models to update the WeightedSum layer for

    step: int
        the intensity of the increase in alpha each turn

    n_steps: int
        number of steps to get from 0 - 1
    """

    alpha = step / float(n_steps - 1)
    for model in models:
        for layer in model.layers:
            if isinstance(layer, WeightedSum):
                backend.set_value(layer.alpha, alpha)


def train_epochs(g_model, d_model, gan_model, dataset, n_epochs, n_batch, fadein=False):
    """Train each model for a given epochs

    Parameters
    ----------
    g_model: keras.models.Model
        generator model

    d_model: keras.models.Model
        discriminator model

    gan_model: keras.models.Model
        composite model
    
    dataset: numpy.ndarray
        source dataset
    
    n_epochs: int
        number of epochs to train over
        
    n_batch: int
        sample batch size
    
    fadein: boolean
        flag denoting if this is a fadein phase of training
    """

    bat_per_epo = int(dataset.shape[0] / n_batch)
    n_steps = bat_per_epo * n_epochs
    
    
    half_batch = int(n_batch / 2)

    for i in range(n_steps):
        if fadein:
            update_fadein([g_model, d_model, gan_model], i, n_steps)

        X_real, y_real = generate_real_samples(dataset, half_batch)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

        d_loss1 = d_model.train_on_batch(X_real, y_real)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)

        z_input = generate_latent_points(latent_dim, n_batch)
        y_real2 = ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(z_input, y_real2)
        # print('>%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))


def scale_dataset(images, new_shape):
    """scale images to a new size

    Parameters
    ----------
    images: list
        source dataset images

    new_shape: tuple
        new images size

    Returns
    -------
    numpy.ndarray
        An array of resized images
    """

    images_list = list()
    for image in images:
        new_image = resize(image, new_shape, 0)
        images_list.append(new_image)
    return asarray(images_list)


def summarize_performance(status, g_model, latent_dim, n_samples=25):
    """generate samples at each phase and save models and plots

    Parameters
    ----------
    status: str
        training phase status

    g_model: keras.models.Model
        generator model
    
    latent_dim:
        dimension of latent points
    
    n_samples:
        number of samples to generate from model
    """

    gen_shape = g_model.output_shape
    name = '%03dx%03d-%s' % (gen_shape[1], gen_shape[2], status)

    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = (X - X.min()) / (X.max() - X.min())
    square = int(sqrt(n_samples))

    for i in range(n_samples):
        pyplot.subplot(square, square, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i])

    filename1 = 'celeba__5000_plot_%s.png' % (name)
    pyplot.savefig(filename1)
    pyplot.close()

    filename2 = 'celeba_5000_model_%s.h5' % (name)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

    

def train(g_models, d_models, gan_models, dataset, latent_dim, e_norm, e_fadein, n_batch):
    """Train the discriminator, generator, and GAN
    
    Parameters
    ----------
    g_models: list
        list of generator models

    d_models: list
        list of discriminator models
    
    gan_modles: list
        list of composit models
    
    dataset: numpy.ndarray
        source dataset
        
    latent_dim: int
        number of latent points
    
    e_norm: int
        training epoch for fine-tuned phase
    
    e_fadin: int
        training epochs for fade-in phase

    n_batch: int
        batch size
    """

    g_normal, d_normal, gan_normal = g_models[0][0], d_models[0][0], gan_models[0][0]
    gen_shape = g_normal.output_shape
    scaled_data = scale_dataset(dataset, gen_shape[1:])
    print('Scaled Data', scaled_data.shape)
    # train normal or straight-through models
    train_epochs(g_normal, d_normal, gan_normal, scaled_data, e_norm[0], n_batch[0])
    summarize_performance('tuned', g_normal, latent_dim)

    for i in range(1, len(g_models)):
        [g_normal, g_fadein] = g_models[i]
        [d_normal, d_fadein] = d_models[i]
        [gan_normal, gan_fadein] = gan_models[i]
        gen_shape = g_normal.output_shape
        scaled_data = scale_dataset(dataset, gen_shape[1:])
        print('Scaled Data', scaled_data.shape)

        # train fade-in models for next level of growth
        train_epochs(g_fadein, d_fadein, gan_fadein, scaled_data, e_fadein[i], n_batch[i], True)
        summarize_performance('faded', g_fadein, latent_dim)

        # train normal or straight-through models
        train_epochs(g_normal, d_normal, gan_normal, scaled_data, e_norm[i], n_batch[i])
        summarize_performance('tuned', g_normal, latent_dim)


In [ ]:
num_growth_phases = 6 # [4, 8, 16, 32, 64, 128]
latent_dims = 100

d_models = define_discriminator(num_growth_phases)
print('d_models defined...')

g_models = define_generator(latent_dims, num_growth_phases)
print('g_models defined...')

gan_models = define_composite(d_models, g_models)
print('gan_models defined...')

dataset = load_real_samples('img_align_celeba_128.npz')
# dataset = load_real_samples('cartoon.npz')

print('Loaded', dataset.shape)

n_batch = [16, 16, 16, 8, 4, 4]
n_epochs = [5, 8, 8, 10, 10, 10]

train(g_models, d_models, gan_models, dataset, latent_dims, n_epochs, n_epochs, n_batch)

### Generate Images from Loaded Model

In [ ]:
def plot_generated(images, n_images):
    """Create a plot of generated images
    
    Parameters
    ----------
    images: numpy.ndarray
        array of images

    n_images: int
        number of images
    """
    square = int(sqrt(n_images))
    images = (images - images.min()) / (images.max() - images.min())

    for i in range(n_images):
        pyplot.subplot(square, square, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(images[i])
    pyplot.show()

In [ ]:
custom_layers = {'PixelNormalization': PixelNormalization, 'MinibatchStdev': MinibatchStdev, 'WeightedSum': WeightedSum}
model = KM.load_model('model_064x064-tuned.h5', custom_layers)

latent_dim = 100
n_images = 25
latent_points = generate_latent_points(latent_dim, n_images)
X  = model.predict(latent_points)
plot_generated(X, n_images)